In [1]:
import pandas
import csv
from pandas.core.frame import DataFrame
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/livialilli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/livialilli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Step 1

In [4]:
rentals_file = pandas.read_csv(("files/Airbnb_Texas_Rentals.csv"), sep =",", delimiter= None, header = "infer", names = None, index_col = None,
                       encoding="ISO-8859-1")

In [69]:
rentals_file.head(3)

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW â...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...


# Step 2

In [5]:
#creating tsv files
i = 0
for r in range(len(rentals_file)):
    record = rentals_file.loc[[r]]
    name = "doc_" + str(i) +".tsv"
    record.to_csv(path_or_buf = name, sep='\t')
    
    i += 1
    

In [39]:
#function to read the tsv file
def read(file_name):
    read_file = pandas.read_csv(file_name, sep ="\t", delimiter= None, header = "infer", names = None, index_col = None, usecols = None,
                       encoding="ISO-8859-1")
    return read_file
    

In [40]:
#for now I just consider one tsv file
d = read("files/doc_0.tsv")

In [41]:
d

,Unnamed: 0,Unnamed: 0.1,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...


In [42]:
d["description"]

0    Welcome to stay in private room with queen bed...
Name: description, dtype: object

In [43]:
d["description"][0]

'Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private bedroom with sofa bed is available for additional guests. 10$ for an additional guest.\\n10min from IAH airport\\nAirport pick-up/drop off is available for $10/trip.'

In [44]:
col_names = ["average_rate_per_night","bedrooms_count","city", "date_of_listing","description","latitude","longitude","title","url"]

# Step 3

## Preprocessing

In [45]:
def stopWords(column_file):
    stopWords = set(stopwords.words('english'))
    words = word_tokenize(str(column_file))
    wordsFiltered = []

    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered
    

In [46]:
def punctuation(list):
    exclude = set(string.punctuation)
    for el in list:
        if el in exclude:
            list.remove(el)
    return list

In [47]:
def stemming(list):
    ps = PorterStemmer()
    output = []
    for word in list:
        stem_word = ps.stem(word)
        output.append(stem_word)
    return output
#it returns a list (of the current column) with all the stem-words


In [48]:
#example of application
s = stopWords(d["title"][0])
p = punctuation(s)
print(stemming(p))

['2', 'privat', 'rooms/bathroom', '10min', 'iah', 'airport']


In [62]:
#On ALL the COLUMNS
#applyng stopWords,punctuation and stemming functions for every column of the dataframe
#it returns a dictionary, where keys are the columns names, and for every key there is the list of cleaned words.
def all_col(read_file):
    dic = {}
    for name in col_names:
        l = stopWords(read_file[name][0])
        l = punctuation(l)
        result = stemming(l)
        dic[name] = result
        #print(result)
    return dic

In [63]:
print(all_col(d))


{'average_rate_per_night': ['27'], 'bedrooms_count': ['2'], 'city': ['humbl'], 'date_of_listing': ['may', '2016'], 'description': ['welcom', 'stay', 'privat', 'room', 'queen', 'bed', 'detach', 'privat', 'bathroom', 'second', 'floor', 'anoth', 'privat', 'bedroom', 'sofa', 'bed', 'avail', 'addit', 'guest', '10', 'addit', 'guest.\\n10min', 'iah', 'airport\\nairport', 'pick-up/drop', 'avail', '10/trip'], 'latitude': ['30.0201379199512'], 'longitude': ['-95.29399600425128'], 'title': ['2', 'privat', 'rooms/bathroom', '10min', 'iah', 'airport'], 'url': ['http', '//www.airbnb.com/rooms/18520444', 'location=cleveland', '2C', '20tx']}


In [67]:
#for every tsv file, I apply the preprocess
for i in range(len(rentals_file)):
    name = "files/doc_" + str(i) +".tsv"
    df = read(name)
    all_col(df)
    